In [ ]:
import os
import numpy as np
import wave
import matplotlib.pyplot as plt
from IPython.display import Audio, display

# Path to the folder
folder_path = '../experiments'

# Get the list of subfolders
subfolders = [f.name for f in os.scandir(folder_path) if f.is_dir()]
ip_mapping = {
    "17221159": "1",
    "17221170": "2",
    "17221157": "3",
    "17221172": "4",
}

def process_audio(experiment_number=None):
    if experiment_number:
        # Process only the specified experiment
        folders_to_process = [f for f in subfolders if experiment_number in f]
    else:
        # Process all experiments
        folders_to_process = subfolders

    for folder in folders_to_process:
        # Create the audio folder if it doesn't exist
        audio_folder_path = os.path.join(folder_path, folder, 'audio_wav_files')
        if not os.path.exists(audio_folder_path):
            os.makedirs(audio_folder_path)

        # Get the list of files in the folder
        files = [f.name for f in os.scandir(audio_folder_path) if f.is_file()]

        # Get the list of files with the .wav extension
        wav_files = [f for f in files if f.endswith('.wav')]

        if len(wav_files) > 0:
            for wav_file in wav_files:
                # Load the audio file
                wav_path = os.path.join(audio_folder_path, wav_file)

                # Extract IP from filename
                ip = wav_file.split("_")[0] 

                # Find the key (IP) from the value
                ip = next((key for key, value in ip_mapping.items() if value == ip), ip)

                # Display the audio
                display(Audio(wav_path))

                # Use wave to read the audio data
                with wave.open(wav_path, 'r') as wf:
                    num_channels = wf.getnchannels()
                    framerate = wf.getframerate()
                    frames = wf.readframes(-1)

                    # Convert the frames to a NumPy array
                    signal = np.frombuffer(frames, dtype=np.int16)

                    # Split the audio into channels if stereo
                    if num_channels == 2:
                        left_channel = signal[::2]
                        right_channel = signal[1::2]
                    else:  # If mono, duplicate the signal for both channels
                        left_channel = right_channel = signal

                # Time axis in seconds
                time = np.linspace(0, len(left_channel) / framerate, num=len(left_channel))

                # Plot the waveforms
                plt.figure(figsize=(10,4))

                plt.subplot(2, 1, 1)
                plt.plot(time, left_channel)
                plt.title(f'Left Channel Waveform of {ip}')
                plt.xlabel('Time (s)')
                plt.ylabel('Amplitude')

                plt.subplot(2, 1, 2)
                plt.plot(time, right_channel)
                plt.title(f'Right Channel Waveform of {ip}')
                plt.xlabel('Time (s)')
                plt.ylabel('Amplitude')

                plt.tight_layout()
                plt.show()
# To process only experiment 'exp_1':
process_audio(experiment_number='exp_1')
# To process all experiments:
# process_audio()